## LlamaIndex

In [ ]:
from llama_index import SQLDatabase, GPTSQLStructStoreIndex
from sqlalchemy import create_engine

engine = create_engine("sqlite:///sns_data.sqlite")
sql_database = SQLDatabase(engine, include_tables=["main_content"])

index = GPTSQLStructStoreIndex(
  [],
  sql_database=sql_database,
  table_name="main_content",
)


In [ ]:
import pandas
response = index.query("玉山", mode="default", verbose = True)
out_df = pandas.DataFrame(eval(response.response) )
out_df

In [ ]:
from llama_index.indices.struct_store import GPTPandasIndex
import os
import pandas

os.environ['OPENAI_API_KEY'] = TOKEN
data_df = pandas.read_excel('ptt_creditcard.xlsx', index_col = 0)

index = GPTPandasIndex(df=data_df)
response = index.query(
  "玉山 or 中信",
  verbose=True
)

## Dashboard

In [ ]:
!pip install jupyter-dash

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("輿情分析儀表板"),
    dcc.Graph(id='graph'),
    html.Label([
        "Query",
        dcc.Input(id='query-input', type='text', value='')
    ]),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("query-input", "value")]
)
def update_figure(Query):
    response = index.query(f"統計每日標題中含有{Query}的文章數量，依照日期排序", mode="default", verbose = True)
    out_df = pandas.read_sql_query(response.extra_info['sql_query'], engine)

    return px.line(out_df, x="date", y="COUNT(*)", 
                   title='sns',render_mode="webgl"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')

In [ ]:
import plotly.express as px
response = index.query("統計每日標題中含有玉山的文章數量，依照日期排序", mode="default", verbose = True)
print(response.extra_info['sql_query'])
our_df = pandas.read_sql_query(response.extra_info['sql_query'], engine)
our_df
fig = px.line(our_df, x="date", y="COUNT(*)", title='sns')
fig.show()